In [3]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import os
os.chdir('D:\\Fraud Analytics\\project')

In [13]:
df = pd.read_csv('NY Property data.csv')

## Filling Missing Values

In [14]:
## Filling NA Zip Codes
df['ZIP'].fillna(0,inplace=True)
hh = df['ZIP'].tolist()
for i in range(len(hh)):
    if hh[i]==0:
        hh[i]=hh[i-1]
df['ZIP'] = hh

In [15]:
def fillgroup(fillvalue,g1='ZIP',g2='TAXCLASS'):
    df['ZT'] = df[g1].astype(int).astype(str)+''+df[g2].astype(str)
## Use median groupby ZIP and TAXCLASS to fill the missing valuables
    fullmedian_zt = df[df[fillvalue] != 0].groupby(['ZT'])[fillvalue].agg({'count':'count', 'median':'median'})
    fullmedian_zt = fullmedian_zt[fullmedian_zt['count']>=10]
## Merge by ZIPTAX
    mer = df.merge(fullmedian_zt, how = 'left', left_on='ZT', right_index=True)
## Fill in NA by ZIPTAX median
    mer[fillvalue].replace(0,np.nan,inplace=True)
    mer[fillvalue].fillna(mer['median'],inplace=True)
    mer[fillvalue].fillna(0,inplace=True)
    fullmedian_t = df[df[fillvalue] != 0].groupby([g2])[fillvalue].agg({'count2':'count', 'median2':'median'})
    mer2 = mer.merge(fullmedian_t, how = 'left', left_on=g2, right_index=True)
    mer2[fillvalue].replace(0,np.nan,inplace=True)
    mer2[fillvalue].fillna(mer2['median2'],inplace=True)
    df[fillvalue]=mer2[fillvalue]

In [16]:
fillgroup('FULLVAL',g1='ZIP',g2='TAXCLASS')

In [17]:
fillgroup('AVTOT',g1='ZIP',g2='TAXCLASS')

In [18]:
fillgroup('AVLAND',g1='ZIP',g2='TAXCLASS')

In [19]:
fillgroup('LTFRONT',g1='ZIP',g2='TAXCLASS')

In [20]:
fillgroup('LTDEPTH',g1='ZIP',g2='TAXCLASS')

In [21]:
fillgroup('BLDFRONT',g1='ZIP',g2='TAXCLASS')

In [22]:
fillgroup('BLDDEPTH',g1='ZIP',g2='TAXCLASS')

In [23]:
fillgroup('STORIES',g1='ZIP',g2='TAXCLASS')

## Building variables

Use the combination of the previous variables to build variables

In [24]:
final_df = df[['RECORD','FULLVAL','AVLAND', 'AVTOT', 'LTFRONT', 'LTDEPTH', 'BLDFRONT', 'BLDDEPTH', 'ZIP', 'STORIES','B','TAXCLASS']]

In [25]:
final_df['S1'] = final_df['LTFRONT'] *final_df['LTDEPTH']
#Area variables by multiplying the Loftfront

In [26]:
final_df['S2'] = final_df['BLDFRONT'] *final_df['BLDDEPTH']

In [27]:
final_df['S3'] = final_df['S2'] *final_df['STORIES']

In [28]:
final_df['r1'] = final_df['FULLVAL'] /final_df['S1']
final_df['r2'] = final_df['FULLVAL'] /final_df['S2']
final_df['r3'] = final_df['FULLVAL'] /final_df['S3']
final_df['r4'] = final_df['AVLAND'] /final_df['S1']
final_df['r5'] = final_df['AVLAND'] /final_df['S2']
final_df['r6'] = final_df['AVLAND'] /final_df['S3']
final_df['r7'] = final_df['AVTOT'] /final_df['S1']
final_df['r8'] = final_df['AVTOT'] /final_df['S2']
final_df['r9'] = final_df['AVTOT'] /final_df['S3']

In [29]:
result = []
for i in final_df['ZIP'].astype(int).astype(str):
    result.append(i[:3])
final_df['ZIP3'] = result

In [30]:
g1 = final_df.groupby('ZIP')[['r1','r2','r3','r4','r5','r6','r7','r8','r9']].mean()
g1.columns = ['r1g1','r2g1','r3g1','r4g1','r5g1','r6g1','r7g1','r8g1','r9g1']

In [31]:
g2 = final_df.groupby('ZIP3')[['r1','r2','r3','r4','r5','r6','r7','r8','r9']].mean()
g2.columns = ['r1g2','r2g2','r3g2','r4g2','r5g2','r6g2','r7g2','r8g2','r9g2']

In [32]:
g3 = final_df.groupby('TAXCLASS')[['r1','r2','r3','r4','r5','r6','r7','r8','r9']].mean()
g3.columns = ['r1g3','r2g3','r3g3','r4g3','r5g3','r6g3','r7g3','r8g3','r9g3']

In [33]:
g4 = final_df.groupby('B')[['r1','r2','r3','r4','r5','r6','r7','r8','r9']].mean()
g4.columns = ['r1g4','r2g4','r3g4','r4g4','r5g4','r6g4','r7g4','r8g4','r9g4']

In [34]:
final_df=final_df.merge(g1,how='left',right_index=True,left_on='ZIP')
final_df=final_df.merge(g2,how='left',right_index=True,left_on='ZIP3')
final_df=final_df.merge(g3,how='left',right_index=True,left_on='TAXCLASS')
final_df=final_df.merge(g4,how='left',right_index=True,left_on='B')

In [35]:
final_df['r1g1']=final_df['r1']/final_df['r1g1']
final_df['r2g1']=final_df['r2']/final_df['r2g1']
final_df['r3g1']=final_df['r3']/final_df['r3g1']
final_df['r4g1']=final_df['r4']/final_df['r4g1']
final_df['r5g1']=final_df['r5']/final_df['r5g1']
final_df['r6g1']=final_df['r6']/final_df['r6g1']
final_df['r7g1']=final_df['r7']/final_df['r7g1']
final_df['r8g1']=final_df['r8']/final_df['r8g1']
final_df['r9g1']=final_df['r9']/final_df['r9g1']
final_df['r1g2']=final_df['r1']/final_df['r1g2']
final_df['r2g2']=final_df['r2']/final_df['r2g2']
final_df['r3g2']=final_df['r3']/final_df['r3g2']
final_df['r4g2']=final_df['r4']/final_df['r4g2']
final_df['r5g2']=final_df['r5']/final_df['r5g2']
final_df['r6g2']=final_df['r6']/final_df['r6g2']
final_df['r7g2']=final_df['r7']/final_df['r7g2']
final_df['r8g2']=final_df['r8']/final_df['r8g2']
final_df['r9g2']=final_df['r9']/final_df['r9g2']
final_df['r1g3']=final_df['r1']/final_df['r1g3']
final_df['r2g3']=final_df['r2']/final_df['r2g3']
final_df['r3g3']=final_df['r3']/final_df['r3g3']
final_df['r4g3']=final_df['r4']/final_df['r4g3']
final_df['r5g3']=final_df['r5']/final_df['r5g3']
final_df['r6g3']=final_df['r6']/final_df['r6g3']
final_df['r7g3']=final_df['r7']/final_df['r7g3']
final_df['r8g3']=final_df['r8']/final_df['r8g3']
final_df['r9g3']=final_df['r9']/final_df['r9g3']
final_df['r1g4']=final_df['r1']/final_df['r1g4']
final_df['r2g4']=final_df['r2']/final_df['r2g4']
final_df['r3g4']=final_df['r3']/final_df['r3g4']
final_df['r4g4']=final_df['r4']/final_df['r4g4']
final_df['r5g4']=final_df['r5']/final_df['r5g4']
final_df['r6g4']=final_df['r6']/final_df['r6g4']
final_df['r7g4']=final_df['r7']/final_df['r7g4']
final_df['r8g4']=final_df['r8']/final_df['r8g4']
final_df['r9g4']=final_df['r9']/final_df['r9g4']

## Z Scaling

In [36]:
X = final_df['r1g1']
from sklearn.preprocessing import scale
r1g1_scale = scale(X)
np.mean(r1g1_scale), np.std(r1g1_scale)

(-1.5712967723201262e-16, 0.9999999999999998)

In [37]:
for i in final_df.columns[15:24].tolist()+final_df.columns[25:].tolist():
    final_df[i] = scale(final_df[i])
    #np.mean(final_df[i]), np.std(final_df[i])

## PCA

In [38]:
pdf = final_df[final_df.columns[15:24].tolist()+final_df.columns[25:].tolist()]

In [39]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pca.fit(pdf)
pca_df = pca.transform(pdf)

In [40]:
pca_df=pd.DataFrame(pca_df)

## Z-scale after PCA

In [41]:
for i in pca_df.columns:
    pca_df[i] = scale(pca_df[i])

## Score 1

In [42]:
pca_df['8']=pca_df.abs().sum(axis=1)

In [43]:
rankpca=pca_df.sort_values(by='8',ascending=True)

In [44]:
rankpca['9']=range(1,1070995)

In [45]:
score = rankpca[['9']]
score.columns=['score1']

## Autoencoder

In [46]:
from sklearn.decomposition import PCA
pca = PCA(n_components=8)
pca.fit(pdf)
pca_df = pca.transform(pdf)
pca_df=pd.DataFrame(pca_df)
for i in pca_df.columns:
    pca_df[i] = scale(pca_df[i])

In [50]:
import pandas as pd
import numpy as np
from scipy import stats
import tensorflow as tf
import matplotlib.pyplot as plt
import seaborn as sns
import pickle
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, precision_recall_curve
from sklearn.metrics import recall_score, classification_report, auc, roc_curve
from sklearn.metrics import precision_recall_fscore_support, f1_score
from sklearn.preprocessing import StandardScaler
from pylab import rcParams
from keras.models import Model, load_model
from keras.layers import Input, Dense
from keras.callbacks import ModelCheckpoint, TensorBoard
from keras import regularizers

Using TensorFlow backend.


In [51]:
nb_epoch = 10
batch_size = 128
input_dim = pca_df.shape[1] #num of columns, 8
encoding_dim = 14
hidden_dim = int(encoding_dim / 2) #i.e. 7
learning_rate = 1e-7

In [52]:
input_layer = Input(shape=(input_dim, ))
encoder = Dense(encoding_dim, activation="tanh", activity_regularizer=regularizers.l1(learning_rate))(input_layer)
encoder = Dense(hidden_dim, activation="relu")(encoder)
decoder = Dense(hidden_dim, activation='tanh')(encoder)
decoder = Dense(input_dim, activation='relu')(decoder)
autoencoder = Model(inputs=input_layer, outputs=decoder)

Instructions for updating:
Colocations handled automatically by placer.


In [53]:
autoencoder.compile(metrics=['accuracy'],
                    loss='mean_squared_error',
                    optimizer='adam')

cp = ModelCheckpoint(filepath="autoencoder_fraud.h5",
                               save_best_only=True,
                               verbose=0)

tb = TensorBoard(log_dir='./logs',
                histogram_freq=0,
                write_graph=True,
                write_images=True)

history = autoencoder.fit(pca_df, pca_df,
                    epochs=nb_epoch,
                    batch_size=batch_size,
                    shuffle=True,
                    validation_data=(pca_df, pca_df),
                    verbose=1,
                    callbacks=[cp, tb]).history

Instructions for updating:
Use tf.cast instead.
Train on 1070994 samples, validate on 1070994 samples
Epoch 1/10
1070994/1070994 [==============================] - 24s 23us/step - loss: 0.9696 - acc: 0.8021 - val_loss: 0.9556 - val_acc: 0.8412
Epoch 2/10
1070994/1070994 [==============================] - 22s 21us/step - loss: 0.9470 - acc: 0.8253 - val_loss: 0.9385 - val_acc: 0.8203
Epoch 3/10
1070994/1070994 [==============================] - 22s 20us/step - loss: 0.9327 - acc: 0.8188 - val_loss: 0.9264 - val_acc: 0.8413
Epoch 4/10
1070994/1070994 [==============================] - 22s 21us/step - loss: 0.9216 - acc: 0.8318 - val_loss: 0.9164 - val_acc: 0.6866
Epoch 5/10
1070994/1070994 [==============================] - 22s 20us/step - loss: 0.9133 - acc: 0.8340 - val_loss: 0.9082 - val_acc: 0.8236
Epoch 6/10
1070994/1070994 [==============================] - 22s 21us/step - loss: 0.9038 - acc: 0.8376 - val_loss: 0.8992 - val_acc: 0.8445
Epoch 7/10
1070994/1070994 [==================

In [54]:
autoencoder = load_model('autoencoder_fraud.h5')

In [55]:
prediction=autoencoder.predict(pca_df)

In [56]:
prediction.shape

(1070994, 8)

In [57]:
diff=prediction-pca_df

In [58]:
diff['8']=diff.abs().sum(axis=1)

In [59]:
rankdiff=diff.sort_values(by='8',ascending=True)

In [60]:
rankdiff['9']=range(1,1070995)

In [61]:
score[2] = rankdiff[['9']]
score.columns=['score1','score2']

In [62]:
score['total']=score['score1']+score['score2']

In [63]:
#By ranking the total score, we consider the highest 20 records as abnormal records
score.sort_values(by='total',ascending=False).head(20)

,score1,score2,total
632815,1070994,1070994,2141988
565391,1070993,1070993,2141986
1067359,1070992,1070992,2141984
917941,1070991,1070991,2141982
585117,1070990,1070990,2141980
85885,1070989,1070988,2141977
585438,1070988,1070989,2141977
565397,1070987,1070987,2141974
585119,1070984,1070986,2141970
556608,1070985,1070985,2141970


In [149]:
score.to_csv('score.csv')